## Imports

In [1]:
import librosa
import spotipy
import os, requests, time, random

import pandas as pd
import numpy as np

from src.obtain.spotify_metadata import generate_token, download_playlist_metadata
from src.vinyl.audio_downloader import download_preview_mp3
from src.vinyl.build_datasets import sample_non_zouk_songs
from src.vinyl.build_datasets import extract_features
from src.vinyl.build_datasets import build_dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import logging
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

Using TensorFlow backend.


# Load New Data, Retrain and Re-evaluate


## Specify Model

In [2]:
features_dict = {
    librosa.feature.mfcc : {'n_mfcc':13},
    librosa.feature.spectral_centroid : {},
    librosa.feature.chroma_stft : {'n_chroma':12},
    librosa.feature.spectral_contrast : {'n_bands':6},
    #librosa.feature.tempogram : {'win_length':192}
}

model_save_path = "models/zouk_classifier_spectral_LSTM3.h5"

## Load Positive Samples

In [3]:
zoukables_metadata_path = 'data/interim/genre_metadata/zoukables_metadata.tsv'
zouk = pd.read_csv(zoukables_metadata_path, sep='\t')
zouk_songs = zouk['id'].tolist()

zouk_features_path = "data/processed/zoukable_spectral.npy"
zouk_data = np.load(zouk_features_path)

## Load Negative Samples

In [4]:
sample_mp3_dir = 'data/raw/mp3s'
metadata_dir = "data/interim/genre_metadata"
genres = os.listdir(metadata_dir)
genres.remove("zoukables_metadata.tsv")

n = zouk.shape[0]
non_zouk_songs, sample_urls = sample_non_zouk_songs(n, genres, metadata_dir)

non_zouk_data = build_dataset(non_zouk_songs, sample_urls, sample_mp3_dir, features_dict)

In [5]:
target = np.array([1] * len(zouk_songs) + [0] * len(non_zouk_songs))

## Build Train/Test Data Sets

In [6]:
X = np.concatenate((zouk_data, non_zouk_data))

train_idx, test_idx, y_train, y_test = train_test_split(
    range(X.shape[0]), target, test_size=0.33, random_state=42, stratify=target)

X_train = X[train_idx,:,:]
X_test = X[test_idx,:,:]

## Load and Retrain Model

In [ ]:
print("Training ...")
batch_size = 35  # num of training examples per minibatch
num_epochs = 400

model = keras.models.load_model(model_save_path)

model.fit(X_train, y_train, batch_size=batch_size, 
          epochs=num_epochs, validation_split=.25, verbose=1,
          callbacks=[
              keras.callbacks.EarlyStopping(patience=8, verbose=1, restore_best_weights=True),
              keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=1),])

model.save(model_save_path)

Training ...




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 597 samples, validate on 200 samples
Epoch 1/400
597/597 [==============================] - 207s 347ms/step - loss: 0.5579 - acc: 0.7320 - val_loss: 0.6210 - val_acc: 0.6750
Epoch 2/400
597/597 [==============================] - 207s 346ms/step - loss: 0.5638 - acc: 0.7152 - val_loss: 0.6210 - val_acc: 0.6750
Epoch 3/400
597/597 [==============================] - 201s 337ms/step - loss: 0.5802 - acc: 0.6951 - val_loss: 0.6210 - val_acc: 0.6750
Epoch 4/400
597/597 [==============================] - 189s 316ms/step - loss: 0.5693 - acc: 0.7085 - val_loss: 0.6209 - val_acc: 0.6750
Epoch 5/400
597/597 [==============================] - 180s 301ms/step - loss: 0.5695 - acc: 0.7085 - val_loss: 0.6208 - val_acc: 0.6750
Epoch 6/400
597/597 [====================

## Evaluate Retrained Model

In [ ]:
print("\nTesting ...")
score, accuracy = model.evaluate(
    X_test, y_test, batch_size=batch_size, verbose=1
)
print("Test loss:  ", score)
print("Test accuracy:  ", accuracy)

### zouk classifier LSTM3 (spectral + tempo):
- `Test Accuracy 1`: 0.7806
- `Test Accuracy 2`: 0.7398
- `Test Accuracy 3`: 0.6990
- `Test Accuracy 4`: 0.7730
- `Test Accuracy 5`: 0.7397
- `Test Accuracy 6`: 0.7449
- `Test Accuracy 7`: 0.7474

### zouk classifier LSTM3 (spectral only):
- `Test Accuracy 1`: 0.8092
- `Test Accuracy 2`: 0.8117
- `Test Accuracy 3`: 0.8168
- `Test Accuracy 4`: 0.7710

### Next Steps
#### More Data
- Start listening to the False Positives and see if they should be included in the Zouk playlist.
- Listen to False Negatives and see if I need to remove anything from Zouk Playlist.
- Make plans to build an app to crowdsource this work.

#### More Models
- LSTMs (add a layer, remove a layer).
- CNNs (including VGG16).
- Convolutional LSTMs.
- XGBoost on Spotify Audio Features.

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_bool = y_pred > 0.65
print(classification_report(y_test, y_pred_bool))

## Inspect Predictions

In [ ]:
all_songs = pd.DataFrame({'song_id':zouk_songs + non_zouk_songs,
                          'target':target})

trainers = all_songs.iloc[train_idx,:].reset_index()

sample0 = trainers[trainers.target==0].sample(10).index
sample1 = trainers[trainers.target==1].sample(10).index
sample_idx = sample0.append(sample1)
samples = trainers.loc[sample_idx]

In [ ]:
y_pred = model.predict(X_train[sample_idx,:])
y_pred_bool = y_pred > 0.75
samples['prediction'] = y_pred_bool.astype(int)
print(classification_report(samples.target, y_pred_bool))

### Listen to False Positives, False Negatives
#### TODO: look up song_id, artist, title

In [ ]:
fp_index = samples[(samples.target==0) & (samples.prediction==1)].index
fn_index = samples[(samples.target==1) & (samples.prediction==0)].index

print("False Positives:")
print("(add these to the list?)")
for i in fp_index:
    filepath = os.path.join(sample_mp3_dir, (samples['song_id'][i] + '.mp3'))
    ipd.display(ipd.Audio(filepath))

print("~" * 32)

print("False Negatives:")
print("(remove these from the list?)")
for i in fn_index:
    filepath = os.path.join(sample_mp3_dir, (samples['song_id'][i] + '.mp3'))
    ipd.display(ipd.Audio(filepath))

# References

- [Keras docs](https://keras.io/)
- [Librosa docs](https://librosa.github.io/librosa/index.html)
- [Spotipy docs](https://spotipy.readthedocs.io)
- [ruohoruotsi: LSTM Music Genre Classification on GitHub](https://github.com/ruohoruotsi/LSTM-Music-Genre-Classification)
- [Music Genre classification using a hierarchical Long Short Term Memory (LSTM) Model](http://www.cs.cuhk.hk/~khwong/p186_acm_00_main_lstm_music_rev5.pdf)
- [Using CNNs and RNNs for Music Genre Recognition](https://towardsdatascience.com/using-cnns-and-rnns-for-music-genre-recognition-2435fb2ed6af) [(GitHub)](https://github.com/priya-dwivedi/Music_Genre_Classification)